# 数据集介绍

来源https://storage.googleapis.com/cluebenchmark/tasks/cluener_public.zip

# 导入包

In [1]:
import torch,json
import sys,os
import pandas as pd
import random
import numpy as np
from nlp_basictasks.tasks import Ner
from nlp_basictasks.evaluation import nerEvaluator
from nlp_basictasks.readers.ner import InputExample

# 获取数据

In [2]:
def _create_examples(input_path,mode):
    examples = []
    with open(input_path, 'r') as f:
        idx = 0
        for line in f:
            json_d = {}
            line = json.loads(line.strip())
            text = line['text']
            label_entities = line.get('label', None)
            words = list(text)
            labels = ['O'] * len(words)
            if label_entities is not None:
                for key, value in label_entities.items():
                    for sub_name, sub_index in value.items():
                        for start_index, end_index in sub_index:
                            assert ''.join(words[start_index:end_index + 1]) == sub_name
                            if start_index == end_index:
                                labels[start_index] = 'S-' + key
                            else:
                                labels[start_index] = 'B-' + key
                                labels[start_index + 1:end_index + 1] = ['I-' + key] * (len(sub_name) - 1)
            json_d['id'] = f"{mode}_{idx}"
            json_d['context'] = " ".join(words)
            json_d['tag'] = " ".join(labels)
            json_d['raw_context'] = "".join(words)
            idx += 1
            examples.append(json_d)
    return examples

In [3]:
data=_create_examples('/data/nfs14/nfs/aisearch/asr/xhsun/datasets/cluener/train.json',mode='train')

In [4]:
len(data)

10748

In [6]:
data[0]

{'id': 'train_0',
 'context': '浙 商 银 行 企 业 信 贷 部 叶 老 桂 博 士 则 从 另 一 个 角 度 对 五 道 门 槛 进 行 了 解 读 。 叶 老 桂 认 为 ， 对 目 前 国 内 商 业 银 行 而 言 ，',
 'tag': 'B-company I-company I-company I-company O O O O O B-name I-name I-name O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'raw_context': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，'}

In [9]:
train_examples=[]
for example in data:
    seq_in=example['context'].strip().split(' ')
    seq_out=example['tag'].strip().split(' ')
    assert len(seq_in)==len(seq_out)
    train_examples.append(InputExample(seq_in=seq_in,seq_out=seq_out))

dev_examples=train_examples[-2000:]
dev_seq_in=[]
dev_seq_out=[]
for example in dev_examples:
    dev_seq_in.append(example.seq_in)
    dev_seq_out.append(example.seq_out)
train_examples=train_examples[:-2000]

In [10]:
# output_path="/data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner"
# if not os.path.exists(os.path.join(output_path,'label2id.json')):
label_set=set()
for examples in data:
    label_list=examples['tag'].strip().split(' ')
    for label in label_list:
        label_set.add(label)
label2id={'[PAD]':0}
for label in label_set:
    label2id[label]=len(label2id)

In [11]:
label2id

{'[PAD]': 0,
 'B-address': 1,
 'S-company': 2,
 'I-book': 3,
 'I-government': 4,
 'B-book': 5,
 'O': 6,
 'I-position': 7,
 'B-scene': 8,
 'S-position': 9,
 'I-movie': 10,
 'S-address': 11,
 'B-game': 12,
 'I-organization': 13,
 'B-government': 14,
 'I-address': 15,
 'B-company': 16,
 'I-scene': 17,
 'B-name': 18,
 'I-company': 19,
 'I-game': 20,
 'I-name': 21,
 'B-movie': 22,
 'B-organization': 23,
 'B-position': 24,
 'S-name': 25}

In [12]:
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/'
ner_model=Ner(model_path,label2id=None,use_crf=True,device='cpu',is_finetune=True)

2021-08-17 09:41:28 - INFO - __init__ - 40 : Load label2id from /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/label2id.json
2021-08-17 09:41:28 - INFO - __init__ - 56 : The label2id is
 {"[PAD]": 0, "I-government": 1, "I-name": 2, "S-name": 3, "B-scene": 4, "S-company": 5, "B-game": 6, "I-company": 7, "O": 8, "I-position": 9, "B-address": 10, "B-name": 11, "B-company": 12, "I-game": 13, "I-movie": 14, "I-book": 15, "B-book": 16, "I-organization": 17, "B-position": 18, "I-scene": 19, "B-organization": 20, "S-address": 21, "B-government": 22, "B-movie": 23, "S-position": 24, "I-address": 25}
2021-08-17 09:41:28 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/BERT/config.json
2021-08-17 09:41:28 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/BERT
2021-08-17 09:41:28 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data

In [12]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
evaluator=nerEvaluator(label2id=label2id,seq_in=dev_seq_in,seq_out=dev_seq_out)

2021-08-16 15:39:05 - INFO - __init__ - 39 : Total evaluate nums : 2000
2021-08-16 15:39:05 - INFO - __init__ - 40 : input is string : False, input is list : True
2021-08-16 15:39:05 - INFO - __init__ - 41 : seq in and out like : 
['并', '且', '更', '有', '动', '力', '练', '习', 'W', 'A', 'R', '3', '，', '为', '一', '战', '成', '名', '而', '准', '备', '着', '，']	['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-game', 'I-game', 'I-game', 'I-game', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
2021-08-16 15:39:05 - INFO - __init__ - 42 : In this evaluator, slot contains ([PAD] I-name I-game S-position B-book S-company I-movie B-scene O B-movie B-game I-position I-organization I-address I-scene I-government B-government B-organization B-company I-company B-name S-name B-address I-book S-address B-position)


In [13]:
ner_model.model.tokenizer.convert_tokens_to_ids(ner_model.model.tokenizer.tokenize("为什么会出现UNJK的问题“?"))

[711,
 784,
 720,
 833,
 1139,
 4385,
 163,
 8171,
 8334,
 8197,
 4638,
 7309,
 7579,
 100,
 136]

In [14]:
ner_model.fit(train_dataloader=train_dataloader,evaluator=evaluator,epochs=5,
              output_path='/data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner')

2021-08-16 15:39:07 - INFO - fit - 163 : 一个epoch 下，每隔54个step会输出一次loss，每隔137个step会评估一次模型


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-08-16 15:39:09 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:
/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [PAD] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        PAD]     0.0000    0.0000    0.0000         0
     address     0.0000    0.0000    0.0000       528
        book     0.0000    0.0000    0.0000       193
     company     0.0000    0.0000    0.0000       521
        game     0.0012    0.0244    0.0023       409
  government     0.0000    0.0000    0.0000       339
       movie     0.0000    0.0000    0.0000       212
        name     0.0000    0.0000    0.0000       700
organization     0.0000    0.0000    0.0000       648
    position     0.0000    0.0000    0.0000       584
       scene     0.0000    0.0000    0.0000       270

   micro avg     0.0002    0.0023    0.0003      4404
   macro avg     0.0001    0.0022    0.0002      4404
weighted avg     0.0001    0.0023    0.0002      4404



...

2021-08-16 16:08:44 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoch 4 after 1 steps:


              precision    recall  f1-score   support

     address     0.5263    0.6250    0.5714       528
        book     0.7285    0.8342    0.7778       193
     company     0.6972    0.7735    0.7334       521
        game     0.7560    0.8411    0.7963       409
  government     0.7179    0.8407    0.7745       339
       movie     0.7895    0.7075    0.7463       212
        name     0.8491    0.9000    0.8738       700
organization     0.7657    0.8071    0.7859       648
    position     0.6979    0.7911    0.7416       584
       scene     0.6026    0.6852    0.6412       270

   micro avg     0.7143    0.7886    0.7496      4404
   macro avg     0.7131    0.7805    0.7442      4404
weighted avg     0.7181    0.7886    0.7510      4404



2021-08-16 16:09:06 - INFO - save_pretrained - 508 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/BERT/pytorch_model.bin
2021-08-16 16:09:06 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/BERT/config.json
2021-08-16 16:09:06 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner/BERT/vocab.txt
2021-08-16 16:09:06 - INFO - fit - 238 : In epoch 4, training_step 0, the eval score is 0.7496222749838117, previous eval score is 0.7436474750723706, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/CompetitionModel/tmp_ner
2021-08-16 16:10:27 - INFO - fit - 216 : Epoch : 4, train_step : 270/1370, loss_value : 0.0989733136224526 
2021-08-16 16:11:49 - INFO - fit - 216 : Epoch : 4, train_step : 540/1370, loss_value : 0.09727128112205753 
2021-08-16 16:12:35 - INFO - __call__ - 59 : nerEvaluator: Evaluating the model on  dataset in epoc

              precision    recall  f1-score   support

     address     0.5189    0.5985    0.5558       528
        book     0.7500    0.8238    0.7852       193
     company     0.7090    0.7716    0.7390       521
        game     0.7345    0.8386    0.7831       409
  government     0.7150    0.8437    0.7740       339
       movie     0.7550    0.7123    0.7330       212
        name     0.8581    0.9071    0.8819       700
organization     0.7419    0.8117    0.7752       648
    position     0.7029    0.8185    0.7563       584
       scene     0.5663    0.6963    0.6246       270

   micro avg     0.7087    0.7911    0.7476      4404
   macro avg     0.7052    0.7822    0.7408      4404
weighted avg     0.7120    0.7911    0.7489      4404



2021-08-16 16:13:32 - INFO - fit - 216 : Epoch : 4, train_step : 810/1370, loss_value : 0.10046491771936417 
2021-08-16 16:14:54 - INFO - fit - 216 : Epoch : 4, train_step : 1080/1370, loss_value : 0.09524496306699735 
2021-08-16 16:16:17 - INFO - fit - 216 : Epoch : 4, train_step : 1350/1370, loss_value : 0.09873104205837956 


In [ ]:
dev_seq_out